In [1]:
!pip install transformers peft bitsandbytes trl deepeval optimum auto-gptq langchain torch flask
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.8/246.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.4/418.4 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 42.0 MB/s eta 0:00:00
     ━━

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig
from trl import SFTTrainer

#Quantize 4bits

In [3]:
compute_dtypes = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtypes,
)

In [11]:
# from google.colab import userdata
# userdata.get('Llama-3-8B')

In [5]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [6]:
# Ensure local directory does not exist
!rm -rf meta-llama/Meta-Llama-3-8B-Instruct

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
access_token = "#"

# Pass the access token to the tokenizer as well
tokenizer = AutoTokenizer.from_pretrained(model_id, token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_id, token = access_token, quantization_config = quant_config,  device_map={"": 0})

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=1000, temperature = 0.7)
hf = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [11]:
prompt_template = PromptTemplate(
    input_variables=["product_name", "product_description", "interest_level", "user_name", "tone"],
    template="""
Write a professional, personalized marketing email in 500 words in a  paragraph form with the following details. Do not generate information about sales, promotions, discounts, prices, discount codes, or dates. Avoid generating restricted words. Ensure the email ends naturally without any additional notes or warnings.

Product Name: {product_name}
Product Description: {product_description}
Interest Level: {interest_level}
User Name: {user_name}

Email Tone: {tone}

Email Content:
Subject: Exclusive {product_name} Offer Just for You!

Dear {user_name},
"""
)

In [12]:
import re

def generate_email_auto(product_name, product_description, interest_level, user_name):
    # Permanent in the template
    sender_name = "Binit Sapkota"
    company_name = "MrBeerGod Technologies"
    website_url = "https://barahsinghe.com/"

    # Determine email tone based on interest level
    tone_map = {
        "very interested": "excited and convincing",
        "interested": "enthusiastic",
        "somewhat interested": "informative",
        "not interested": "gentle reminder"
    }
    tone = tone_map.get(interest_level, "informative")

    # Prepare the input for the LLMChain
    input_data = {
        "product_name": product_name,
        "product_description": product_description,
        "interest_level": interest_level,
        "user_name": user_name,
        "tone": tone
    }

    generation_result = email_chain.invoke(input_data)   # invoke should be used instead of .run in runnable
    email_content = generation_result.strip()

    forbidden_phrases = [
        "code", "coupon code", "discount code", "offer code", "Discount Offer", "Your", "in paragraph", "Name",
        "Social Media Handle", "Physical Address", "City, State, ZIP", "Country", "Time Zone", "Language",
        "Currency", "Region", "Industry", "Size", "Job", "Type", "Business Model", "Revenue", "Employee Count", "Website",
        "Company", "Social Media Links", "Website", "LinkedIn Profile", "Twitter Handle","Facebook Page", "endblock", "block email", "block content",
        "retailed price", "retailed rate", "discounted price", "discounted rate",   "Email Address", "Phone Number",
        "price", "product price", "30-day money-back guarantee", "30% off", "20% off", "Your Name", "Your Company",
        "15% off", "10% off", "Regards,", "Best regards,", "Sincerely,", "Yours truly,", "Best", 'title', 'Company logo',
        "money would be refunded", "company would be liable", "product can be returned after used", "simply use the discount",
        "Email Signature" , "Your Title" , "Name of Company", "Company Name", "Contact Information", "Email length", "Email Format", "Email Tone",
        "Email Content",  "250 words", "paragraphs", "Excited and convincing", "Personalized and professional", "Email Purpose: To generate interest and curiosity about the SuperWidget",
        "Note I'll make sure to avoid generating information about sales, promotions, discounts, s, s, or dates. Also, I'll keep the tone , and the content will be  form. Let me know if you need any further adjustments!"




    ]
    forbidden_symbols = r'https|www|\\|~|`|<|>|[\[\]%\*#|:[]{}|]'

    # Join all the phrases with '|' to create a regex pattern
    pattern = '|'.join(map(re.escape, forbidden_phrases))

    # Substitute any occurrences of the phrases with an empty string
    email_content = re.sub(pattern, '', email_content, flags=re.IGNORECASE)
    email_content = re.sub(forbidden_symbols, '', email_content, flags=re.IGNORECASE)

    personalized_email = f"""
{email_content}

Best regards,
{sender_name}
{company_name}
Visit us: {website_url}
"""

    return personalized_email


product_name = "Budweiser beer"
product_description = "Budweiser beer is a crisp and refreshing lager known for its iconic red label and smooth, balanced taste."
interest_level = "very interested"
user_name = "Shahrukh Khan"

personalized_email = generate_email_auto(product_name, product_description, interest_level, user_name)
print(personalized_email)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Write one professional personalized marketing email in 500 words a paragraph form with the following details but do not generate information about sales, promotions, discounts, s, s, or dates. dont generate restricted words:
and don't generate note at the last line. Only generate email dont generate other unnecessary things.

Product : Budweiser beer
Product Description: Budweiser beer is a crisp and refreshing lager known for its iconic red label and smooth, balanced taste.
Interest Level: very interested
User : Shahrukh Khan

: 

 :
Subject: Exclusive Budweiser beer Offer Just for You!

Dear Shahrukh Khan,
I'm thrilled to share with you an extraordinary opportunity to indulge in the finest Budweiser beer experience! As someone who has shown a remarkable interest in our craft, I'm excited to introduce you to the world of Budweiser beer, where every sip is a celebration of flavor and joy. Our iconic red label beer is renowned for its crisp and refreshing lager, perfect for any occasio